# Lecture 2: Quantifying Uncertainties in Physical Models

> Ignorance is preferable to error and he is less remote from the truth who believes nothing than he who believes what is wrong.
Thomas Jefferson (1781)

## Objectives
+ To tell the difference between **aleatory** and **epistemic** uncertainties.
+ To define **predictive modeling**.
+ To use **probability theory** to represent both aleatory and epistemic uncertainties.
+ To **propagate uncertainty** through a physical model using Monte Carlo.

## Readings

+ [Oden, Moser, Ghattas, Computer Predictions with Quantified Uncertainty, Part I](https://archive.siam.org/pdf/news/1842.pdf)

+ [Oden, Moser, Ghattas, Computer Predictions with Quantified Uncertainty, Part II](https://archive.siam.org/pdf/news/1857.pdf)

## Definitions

We are not going to make a big effort to be consistent about the use of the following terms, since their precise meaning is still under debate.

### Uncertainty
In general, we are uncertain about a logical proposition if we do not know whether it is true or false. 
In particular, we can be uncertain about:
+ the value of a model parameter;
+ the mathematical form of a model;
+ the initial conditions of a ordinary differntial equations;
+ the boundary conditions of a partial differential equation;
+ the value of an experimental measurment we are about to perform;
+ etc.

Uncertainty may be *aleatory* or *epistemic*. Aleatory uncertainty is associated with inherent system randomness. Epistemic uncertainty is associated with lack of knowledge. If you think too hard, the distinction between the two becomes philosophical. We are not going to push this too hard. Fortunately, our approach (the Bayesian approach) treats both uncertainties on an equal footing.

### Predictive Modeling
*Predictive modeling* is the process of assigning error bars to the predictions of computational models.
Ideally, these error bars rigorously quantify the effect of all associated uncertainties.
Having quantified and propagated uncertainties through the computational models, one can assess the risk of making decisions based on the model predictions.


## Example: Catalytic Conversion of Nitrate to Nitrogen

This is Example 3.1 of [(Tsilifis, 2014)](http://arxiv.org/abs/1410.5522).

Consider the catalytic
conversion of nitrate ($\mbox{NO}_3^-$) to nitrogen ($\mbox{N}_2$) and other
by-products by electrochemical means.
The mechanism that is followed is complex and not well understood.
The experiment of [(Katsounaros, 2012)](http://www.sciencedirect.com/science/article/pii/S0013468612005208) confirmed the
production of nitrogen ($\mbox{N}_2$), ammonia
($\mbox{NH}_3$), and nitrous oxide ($\mbox{N}_2\mbox{O}$) as final products
of the reaction, as well as the intermediate production of nitrite ($\mbox{NO}_2^-$).
The data are reproduced in [Comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values) (CSV) and stored in
[data/catalysis.csv](data/catalysis.csv).
The time is measured in minutes and the conentrations are measured in $\mbox{mmol}\cdot\mbox{L}^{-1}$.
Let's load the data into this notebook using the [Pandas](http://pandas.pydata.org) Python module:


In [ ]:
# If this fails, you haven't uploaded "catalysis.csv".
# Repeat 11 of the instructions.
import pandas as pd
catalysis_data = pd.read_csv('catalysis.csv', index_col=0)
catalysis_data

Let's visualize the data using [Matplotlib](http://matplotlib.org):

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
catalysis_data.plot()

The theory of catalytic reactions guarantees that the total mass must be conserved.
However, this is not the case in our dataset:

In [ ]:
catalysis_data.sum(axis=1)

This inconsistency suggests the existence of an intermediate unobserved reaction product X.
[(Katsounaros, 2012)](http://www.sciencedirect.com/science/article/pii/S0013468612005208) suggested that the following reaction path shown in the following figure.

![](scheme.png "Reaction Scheme")

The dynamical system associated with the reaction is:
$$
\begin{array}{cc}
\frac{d \left[\mbox{NO}_3^-\right]}{dt} &= -k_1\left[\mbox{NO}_3^-\right], \\
\frac{d\left[\mbox{NO}_2^-\right]}{dt} &= k_1\left[\mbox{NO}_3^-\right] - (k_2 + k_4 +
k_5)[\mbox{NO}_2^-], \\
\frac{d \left[\mbox{X}\right]}{dt} &= k_2 \left[\mbox{NO}_2^-\right] - k_3 [X],\\
\frac{d \left[\mbox{N}_2\right]}{dt} &= k_3 \left[\mbox{X}\right], \\
\frac{d \left[\mbox{NH}_3\right]}{dt} &= k_4 \left[\mbox{NO}_2^-\right],\\
\frac{d \left[\mbox{N}_2O\right]}{dt} &= k_5 \left[\mbox{NO}_2^-\right],
\end{array}
$$
where $[\cdot]$ denotes the concentration of a quantity, and
$k_i > 0$, $i=1,...5$ are the *kinetic rate constants*.


### Questions 01

<span><a class="pd-embed" id="pd1514456211022" href="http://ebilionis.polldaddy.com/s/handout-01-01">Click here to respond</a></span>
<script type="text/javascript">
var _polldaddy = [] || _polldaddy;

_polldaddy.push( {
    type: 'button',
    title: 'Take Our Survey!',
    style: 'inline',
    domain: 'ebilionis.polldaddy.com/s/',
    id: 'handout-01-01',
    placeholder: 'pd1514456211022'
} );

(function(d,c,j){if(!document.getElementById(j)){var pd=d.createElement(c),s;pd.id=j;pd.src=('https:'==document.location.protocol)?'https://polldaddy.com/survey.js':'http://i0.poll.fm/survey.js';s=document.getElementsByTagName(c)[0];s.parentNode.insertBefore(pd,s);}}(document,'script','pd-embed'));
</script>

### Computational Model

We will develop a generic computational model for the solution of dynamical systems and we will use it to study the catalysis problem. The code relies on the [Fourth-order Runge-Kutta method](https://en.wikipedia.org/wiki/Runge–Kutta_methods) and is a modified copy of [http://www.math-cs.gordon.edu/courses/ma342/python/diffeq.py](http://www.math-cs.gordon.edu/courses/ma342/python/diffeq.py) developed by Jonathan Senning. The code solves:

$$
\begin{array}{ccc}
\dot{\mathbf{y}} &=& f(\mathbf{y}, t),\\
\mathbf{y}(0) &=& \mathbf{y}_0.
\end{array}
$$

In [ ]:
import numpy as np
def rk45( f, y0, t, args=() ):
    """Fourth-order Runge-Kutta method with error estimate.

    USAGE:
        y = rk45(f, x0, t, args=())

    INPUT:
        f     - function of x and t equal to dx/dt.  x may be multivalued,
                in which case it should a list or a NumPy array.  In this
                case f must return a NumPy array with the same dimension
                as x.
        y0    - the initial condition(s).  Specifies the value of x when
                t = t[0].  Can be either a scalar or a list or NumPy array
                if a system of equations is being solved.
        t     - list or NumPy array of t values to compute solution at.
                t[0] is the the initial condition point, and the difference
                h=t[i+1]-t[i] determines the step size h.
        args  - any other parameters of the function f.

    OUTPUT:
        y     - NumPy array containing solution values corresponding to each
                entry in t array.  If a system is being solved, x will be
                an array of arrays.

    NOTES:
        This version is based on the algorithm presented in "Numerical
        Mathematics and Computing" 6th Edition, by Cheney and Kincaid,
        Brooks-Cole, 2008.
    """

    # Coefficients used to compute the independent variable argument of f

    c20  =   2.500000000000000e-01  #  1/4
    c30  =   3.750000000000000e-01  #  3/8
    c40  =   9.230769230769231e-01  #  12/13
    c50  =   1.000000000000000e+00  #  1
    c60  =   5.000000000000000e-01  #  1/2

    # Coefficients used to compute the dependent variable argument of f

    c21 =   2.500000000000000e-01  #  1/4
    c31 =   9.375000000000000e-02  #  3/32
    c32 =   2.812500000000000e-01  #  9/32
    c41 =   8.793809740555303e-01  #  1932/2197
    c42 =  -3.277196176604461e+00  # -7200/2197
    c43 =   3.320892125625853e+00  #  7296/2197
    c51 =   2.032407407407407e+00  #  439/216
    c52 =  -8.000000000000000e+00  # -8
    c53 =   7.173489278752436e+00  #  3680/513
    c54 =  -2.058966861598441e-01  # -845/4104
    c61 =  -2.962962962962963e-01  # -8/27
    c62 =   2.000000000000000e+00  #  2
    c63 =  -1.381676413255361e+00  # -3544/2565
    c64 =   4.529727095516569e-01  #  1859/4104
    c65 =  -2.750000000000000e-01  # -11/40

    # Coefficients used to compute 4th order RK estimate

    a1  =   1.157407407407407e-01  #  25/216
    a2  =   0.000000000000000e-00  #  0
    a3  =   5.489278752436647e-01  #  1408/2565
    a4  =   5.353313840155945e-01  #  2197/4104
    a5  =  -2.000000000000000e-01  # -1/5

    b1  =   1.185185185185185e-01  #  16.0/135.0
    b2  =   0.000000000000000e-00  #  0
    b3  =   5.189863547758284e-01  #  6656.0/12825.0
    b4  =   5.061314903420167e-01  #  28561.0/56430.0
    b5  =  -1.800000000000000e-01  # -9.0/50.0
    b6  =   3.636363636363636e-02  #  2.0/55.0

    n = len( t )
    y = np.array( [ y0 ] * n )
    for i in range( n - 1 ):
        h = t[i+1] - t[i]
        k1 = h * f( y[i], t[i], *args )
        k2 = h * f( y[i] + c21 * k1, t[i] + c20 * h, *args )
        k3 = h * f( y[i] + c31 * k1 + c32 * k2, t[i] + c30 * h, *args )
        k4 = h * f( y[i] + c41 * k1 + c42 * k2 + c43 * k3, t[i] + c40 * h, *args )
        k5 = h * f( y[i] + c51 * k1 + c52 * k2 + c53 * k3 + c54 * k4, \
                        t[i] + h, *args )
        k6 = h * f( \
            y[i] + c61 * k1 + c62 * k2 + c63 * k3 + c64 * k4 + c65 * k5, \
            t[i] + c60 * h, *args )

        y[i+1] = y[i] + a1 * k1 + a3 * k3 + a4 * k4 + a5 * k5
        y5 = y[i] + b1 * k1 + b3 * k3 + b4 * k4 + b5 * k5 + b6 * k6

    return y

## Calibrating the Catalysis Model to the Experimental Data

Now that we are certain that our generic ODE solver works, let us use it to develop a solver for the catalysis model. All, we need to do is define the right hand side of the dynamics:

In [ ]:
def f_catalysis(y, t, kappa):
    rhs = np.zeros((6,))
    rhs[0] = -kappa[0] * y[0]
    rhs[1] = kappa[0] * y[0] - (kappa[1] + kappa[3] + kappa[4]) * y[1]
    rhs[2] = kappa[1] * y[1] - kappa[2] * y[2]
    rhs[3] = kappa[2] * y[2]
    rhs[4] = kappa[3] * y[1]
    rhs[5] = kappa[4] * y[1]
    return rhs

Let's try to calibrate the parameters of the model to the data, manually. Because the parameters are too small, let us work with the transformed version:

$$
\xi_i = \log\left(180k_i\right).
$$

In [ ]:
from ipywidgets import interactive
def compare_model_to_data(xi1 = 1.359, xi2 = 1.657, xi3 = 1.347, xi4 = -.162, xi5 = -1.009):
    """
    Compare the model predictions to the data.
    """
    t = np.linspace(0, 180, 100)
    kappa = np.exp([xi1, xi2, xi3, xi4, xi5]) / 180.
    y = rk45(f_catalysis, (500., 0., 0., 0., 0., 0.), t, args=(kappa,))
    fig, ax = plt.subplots(figsize=(10, 10))
    catalysis_data.plot(ax=ax, style='s')
    ax.plot(t, y[:, 0], color=sns.color_palette()[0], label='Model NO3-')
    ax.plot(t, y[:, 1], color=sns.color_palette()[1], label='Model NO2-')
    ax.plot(t, y[:, 2], color=sns.color_palette()[5], label='Model X')
    ax.plot(t, y[:, 3], color=sns.color_palette()[2], label='Model N2')
    ax.plot(t, y[:, 4], color=sns.color_palette()[3], label='Model NH3')
    ax.plot(t, y[:, 5], color=sns.color_palette()[4], label='Model N2O')
    plt.legend()
    
interactive(compare_model_to_data, xi1 = (-2, 2, 0.05), xi2 = (-2, 2, 0.05), xi3 = (-2, 2, 0.05),
                                   xi4 = (-2, 2, 0.05), xi5 = (-2, 2, 0.05) )

This is the calibration problem. 

### Questions 02

<span><a class="pd-embed" id="pd1514457269730" href="http://ebilionis.polldaddy.com/s/handout-01-02">Click to respond.</a></span>
<script type="text/javascript">
var _polldaddy = [] || _polldaddy;

_polldaddy.push( {
    type: 'button',
    title: 'Click to respond.',
    style: 'inline',
    domain: 'ebilionis.polldaddy.com/s/',
    id: 'handout-01-02',
    placeholder: 'pd1514457269730'
} );

(function(d,c,j){if(!document.getElementById(j)){var pd=d.createElement(c),s;pd.id=j;pd.src=('https:'==document.location.protocol)?'https://polldaddy.com/survey.js':'http://i0.poll.fm/survey.js';s=document.getElementsByTagName(c)[0];s.parentNode.insertBefore(pd,s);}}(document,'script','pd-embed'));
</script>

### Uncertainty Propagation

As discussed in Question 2 above, there various reasons why a model cannot be calibrated perfectly. Some of these are:

+ lack of data;
+ the existence of measurement noise;
+ the fact that the model is just not perfect.

Ignoring for the moment the possibility that the model is just bluntly wrong, we see that the lack of data or the presence of noise will induce some uncertainty in the values of the calibrated parameters. We are going to represent uncertainty on parameters by assigning a probability density on them. There are systematic ways of estimating the uncertainty induced because of the calibration process, but this will not concern us now.
For the moment, assume that somebody told us that the uncertainty in the scaled parameters $\xi_i$ of the model is as follows:


| Variable | Value |
|---------|------------------|
| $\xi_1$ |$1.35\pm 0.05$ |
| $\xi_2$ |$1.65\pm 0.08$   |
| $\xi_3$ |$1.34\pm 0.11$ |
| $\xi_4$ |$-0.16\pm 0.16$ |
| $\xi_5$ |$-3.84\pm 0.20$ |

But what does this information actually mean? As we will discuss in the following lectures, this information can be used to assign a probability density on each one of these parameters, say $p(\xi_i)$, that *models* our state of knowledge about them. For example, let us assume that our state of knowledge about $\xi_1$ is given by a Gaussian probability density:

$$
p(\xi_1) = \mathcal{N}(\xi_1|\mu_1=1.35, \sigma^2 = 0.05^2),
$$

which we can visualize as follows:

In [ ]:
import scipy.stats
from scipy.stats import norm
xi1 = np.linspace(-0, 2, 200)
plt.plot(xi1, norm.pdf(xi1, loc=1.35, scale=0.05))

This means that we do not beleive that the value of the parameter can be less than 1.0 or greater than 1.6. Note that, we are deliberately trying to avoid the use of the term "random". There is nothing random in our example. Probability models a state of knowledge.

How does this uncertainty propagate through the model? We will study this question with a simple numerical experiment. We are going to assign Gaussian probability densities on all the $\xi_i$'s, sample them a few times, and run our catalysis model for each one.

In [ ]:
def plot_samples(mu1 = 1.359, sig1=0.055,
                mu2 = 1.657, sig2=0.086,
                mu3 = 1.347, sig3=0.118,
                mu4 = -.162, sig4=0.167,
                mu5 = -1.009, sig5=0.368,
                num_samples=1):
    """
    Take a few samples of the model to study uncertainty propagation.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    catalysis_data.plot(ax=ax, style='s')
    t = np.linspace(0, 180, 100)
    for i in range(num_samples):
        xi1 = norm.rvs(loc=mu1, scale=sig1)
        xi2 = norm.rvs(loc=mu2, scale=sig2)
        xi3 = norm.rvs(loc=mu3, scale=sig3)
        xi4 = norm.rvs(loc=mu4, scale=sig4)
        xi5 = norm.rvs(loc=mu5, scale=sig5)
        kappa = np.exp([xi1, xi2, xi3, xi4, xi5]) / 180.
        y = rk45(f_catalysis, (500., 0., 0., 0., 0., 0.), t, args=(kappa,))
        ax.plot(t, y[:, 0], linewidth=0.5, color=sns.color_palette()[0])#, label='Model NO3-')
        ax.plot(t, y[:, 1], linewidth=0.5, color=sns.color_palette()[1])#, label='Model NO2-')
        ax.plot(t, y[:, 2], linewidth=0.5, color=sns.color_palette()[5])#, label='Model X')
        ax.plot(t, y[:, 3], linewidth=0.5, color=sns.color_palette()[2])#, label='Model N2')
        ax.plot(t, y[:, 4], linewidth=0.5, color=sns.color_palette()[3])#, label='Model NH3')
        ax.plot(t, y[:, 5], linewidth=0.5, color=sns.color_palette()[4])#, label='Model N2O')
    plt.legend()

interactive(plot_samples, mu1 = (-2, 2, 0.05), sig1=(0.02, 0.4, 0.01),
                                   mu2 = (-2, 2, 0.05), sig2=(0.02, 0.4, 0.01),
                                   mu3 = (-2, 2, 0.05), sig3=(0.02, 0.4, 0.01),
                                   mu4 = (-2, 2, 0.05), sig4=(0.02, 0.4, 0.01),
                                   mu5 = (-2, 2, 0.05), sig5=(0.02, 0.4, 0.01),
            num_samples=(1, 1100, 10))

### Questions 03

Increase the number of samples from 1, to 10, to 100, to 1000. Each time you get a better description of uncertainty. This is a Monte Carlo simulation. Then, take the survey.


<span><a class="pd-embed" id="pd1514457576895" href="http://ebilionis.polldaddy.com/s/handout-01-03">Click here to respond.</a></span>
<script type="text/javascript">
var _polldaddy = [] || _polldaddy;

_polldaddy.push( {
    type: 'button',
    title: 'Click here to respond.',
    style: 'inline',
    domain: 'ebilionis.polldaddy.com/s/',
    id: 'handout-01-03',
    placeholder: 'pd1514457576895'
} );

(function(d,c,j){if(!document.getElementById(j)){var pd=d.createElement(c),s;pd.id=j;pd.src=('https:'==document.location.protocol)?'https://polldaddy.com/survey.js':'http://i0.poll.fm/survey.js';s=document.getElementsByTagName(c)[0];s.parentNode.insertBefore(pd,s);}}(document,'script','pd-embed'));
</script>